In [49]:
#import pandas and os
import pandas as pd
import os

## Merging 12 months of data in single file

In [50]:
# test if data url works
# df = pd.read_csv("Sales_Data/Sales_April_2019.csv")
# df.head()

In [51]:
files = [file for file in os.listdir('Sales_Data')]

all_months_data =  pd.DataFrame()

for file in files:
    if file != ".DS_Store":
#         print(file)
        df = pd.read_csv("Sales_Data/"+file)
        all_months_data = pd.concat([all_months_data, df])

In [52]:
all_months_data.to_csv("all_data.csv")

In [53]:
all_data = pd.read_csv("all_data.csv")
all_data.shape

(186850, 7)

## Data Cleanup

### Find all NaN

In [63]:
row_has_NaN = all_data.isnull().any(axis=1)
rows_with_NaN = all_data[row_has_NaN]
rows_with_NaN.shape
all_data = all_data.dropna(how='any')
all_data.head(2)

,Unnamed: 0,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"


*** Found 545 row out of all_data containing NaN and removed ***